In [2]:
import duckdb
import pandas as pd
import config

data = duckdb.connect(f"{config.DATA_DIR_NAME}/data_tables.db")

## School type

In [3]:
data.sql("""
    select *
    from sen_phase_type
    limit 5
    """)

┌─────────────┬─────────────────┬──────────────────┬──────────────┬──────────────┬───────────────┬─────────────┬─────────────┬───────────────┬─────────────┬─────────────────────────────┬─────────────────────────────┬─────────────────┬──────────────┬─────────────┬──────────┬─────────────────────┬──────────────────┐
│ time_period │ time_identifier │ geographic_level │ country_code │ country_name │  region_name  │ region_code │ old_la_code │    la_name    │ new_la_code │     phase_type_grouping     │    type_of_establishment    │ hospital_school │ total_pupils │ sen_support │ ehc_plan │ sen_support_percent │ ehc_plan_percent │
│    int64    │     varchar     │     varchar      │   varchar    │   varchar    │    varchar    │   varchar   │    int64    │    varchar    │   varchar   │           varchar           │           varchar           │     varchar     │   varchar    │   varchar   │ varchar  │       varchar       │     varchar      │
├─────────────┼─────────────────┼──────────────────┼

In [4]:
# query into a pandas dataframe
school_type = data.sql(
    """
    select *
    from sen_phase_type
    """).df()

school_type.head()

,time_period,time_identifier,geographic_level,country_code,country_name,region_name,region_code,old_la_code,la_name,new_la_code,phase_type_grouping,type_of_establishment,hospital_school,total_pupils,sen_support,ehc_plan,sen_support_percent,ehc_plan_percent
0,202324,Academic year,Local authority,E92000001,England,East Midlands,E12000004,925,Lincolnshire,E10000019,State-funded secondary,Academy sponsor led,No,16348,2842,615,17.38438953,3.761928065
1,202324,Academic year,Local authority,E92000001,England,East Midlands,E12000004,925,Lincolnshire,E10000019,State-funded secondary,Community school,No,2111,173,7,8.195168167,0.3315964
2,202324,Academic year,Local authority,E92000001,England,West Midlands,E12000005,335,Walsall,E08000030,State-funded special school,Academy special sponsor led,No,140,0,140,0,100
3,202324,Academic year,Local authority,E92000001,England,West Midlands,E12000005,335,Walsall,E08000030,State-funded special school,Community special school,No,790,0,790,0,100
4,202324,Academic year,Local authority,E92000001,England,West Midlands,E12000005,336,Wolverhampton,E08000031,Independent school,Other independent school,No,1430,238,11,16.64335664,0.769230769


## Age groups

In [5]:
data.sql("""
    select *
    from sen_age_sex
    limit 5
    """)

┌─────────────┬─────────────────┬──────────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────┬─────────────┬───────────────────────────────┬──────────────────┬────────────────────────────┬─────────────────┬─────────────────┬─────────────┬───────────────┐
│ time_period │ time_identifier │ geographic_level │ country_code │ country_name │ region_name │ region_code │ old_la_code │ la_name │ new_la_code │      phase_type_grouping      │ pupil_sen_status │        primary_need        │ breakdown_topic │    breakdown    │ pupil_count │ pupil_percent │
│    int64    │     varchar     │     varchar      │   varchar    │   varchar    │   varchar   │   varchar   │    int64    │ varchar │   varchar   │            varchar            │     varchar      │          varchar           │     varchar     │     varchar     │    int64    │    double     │
├─────────────┼─────────────────┼──────────────────┼──────────────┼──────────────┼─────────────┼─────────────┼─────

In [6]:
# query into a pandas dataframe
age_groups = data.sql(
    """
    select time_period, phase_type_grouping, breakdown, sum(pupil_count) as pupil_count
    from sen_age_sex
    where phase_type_grouping = 'State-funded primary' and pupil_sen_status = 'EHC plans' and breakdown_topic = 'Age'
    group by time_period, pupil_sen_status, phase_type_grouping, breakdown
    order by breakdown, time_period    
    """).df()

age_groups.head()

,time_period,phase_type_grouping,breakdown,pupil_count
0,201516,State-funded primary,Age 10,64980.0
1,201617,State-funded primary,Age 10,64356.0
2,201718,State-funded primary,Age 10,69480.0
3,201819,State-funded primary,Age 10,79254.0
4,201920,State-funded primary,Age 10,89454.0


## Year groups

In [6]:
data.sql("""
    select *
    from sen_ncyear
    limit 5
    """)

┌─────────────┬─────────────────┬──────────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────┬─────────────┬───────────────────────────────┬──────────────────┬─────────────────────────────┬──────────────────┬────────────────┬──────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┬─────────────────┬────────────────────────┬──────────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬─────────────────────────┐
│ time_period │ time_identifier │ geographic_level │ country_code │ country_name │ region_name │ region_code │ old_la_code │ la_name │ new_la_code │      phase_type_grou

In [14]:
# query into a pandas dataframe
year_groups = data.sql(
    """
    select  time_period, 
            la_name, 
            primary_need,
            sum(number_of_pupils) as number_of_pupils, 
            sum(nc_early_years) as nc_early_years, 
            sum(nc_reception) as nc_reception,
            sum(nc_year_1) as nc_year_1,
            sum(nc_year_2) as nc_year_2,
            sum(nc_year_3) as nc_year_3,
            sum(nc_year_4) as nc_year_4,
            sum(nc_year_5) as nc_year_5,
            sum(nc_year_6) as nc_year_6,
            sum(nc_year_7) as nc_year_7,
            sum(nc_year_8) as nc_year_8,
            sum(nc_year_9) as nc_year_9,
            sum(nc_year_10) as nc_year_10,
            sum(nc_year_11) as nc_year_11,
            sum(nc_year_12) as nc_year_12,
            sum(nc_year_13) as nc_year_13,
            sum(nc_year_14) as nc_year_14
    from sen_ncyear
    group by time_period, la_name, primary_need
    """).df()

year_groups.head()

,time_period,la_name,primary_need,number_of_pupils,nc_early_years,nc_reception,nc_year_1,nc_year_2,nc_year_3,nc_year_4,nc_year_5,nc_year_6,nc_year_7,nc_year_8,nc_year_9,nc_year_10,nc_year_11,nc_year_12,nc_year_13,nc_year_14
0,201617,Isle of Wight,Missing,55064.0,0.0,4980.0,4576.0,4460.0,4204.0,4280.0,4404.0,4384.0,4248.0,4020.0,3816.0,3480.0,3988.0,2328.0,1892.0,4.0
1,201617,East Sussex,Moderate Learning Difficulty,4460.0,4.0,80.0,272.0,332.0,368.0,460.0,512.0,588.0,344.0,356.0,340.0,308.0,408.0,52.0,16.0,20.0
2,201617,Hampshire,Other Difficulty/Disability,2856.0,16.0,156.0,132.0,268.0,196.0,136.0,140.0,136.0,408.0,292.0,400.0,316.0,252.0,4.0,4.0,0.0
3,201617,Hampshire,Profound & Multiple Learning Difficulty,612.0,24.0,44.0,52.0,20.0,56.0,52.0,44.0,36.0,48.0,52.0,60.0,32.0,32.0,32.0,28.0,0.0
4,201617,Hampshire,Severe Learning Difficulty,3252.0,88.0,176.0,208.0,288.0,292.0,264.0,260.0,180.0,212.0,180.0,216.0,192.0,196.0,160.0,204.0,136.0
